In [3]:
import numpy as np 
import pandas as pd

In [4]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv('iris.csv', skiprows=1,header=None,names=col_names)
data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [5]:
data['type'] = data['type'].astype('category')

In [6]:
data['type'] = data['type'].cat.codes

In [7]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor'''
        #for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # for leaf node
        self.value = value

In [9]:
class DecisionTreeClassifier():
    def __init__(self, min_sample_split=2, max_depth=2):
        '''contructor'''

        #initialize the root of the tree
        self.root = None

        #stooping conditions
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        """recursive function to build the tree"""  

        X, Y = dataset[:,:-1], dataset

    
    def get_best_split(self, dataset, num_samples, num_features):
        """ function to find the best split"""

        #dictionary to store the best split
        best_split = {}
        max_infor_gain = -float("inf")

        #loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index] 
            possible_thresholds = np.unique(feature_index)
            #podriamos ir sobre todos los numero reales entre el max y min pero para resumir vamos solo sobre los valores que tome esa feature
            #loop over all the features values presnet in the data
            for threshold in possible_thresholds:
                #get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if the chulds are not null
                #if len(dataset_left) 

    def split(self, dataset, feature_index, threshold):
        '''function to split the data'''

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_righ = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_righ
    
    def information_gain(self, parent, l_child, r_child, mode='entropy'):
        '''function to compute information gain'''

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=='gini':
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child)) + weight_r*self.gini_index(r_child)
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child)) + weight_r*self.entropy(r_child)
        return gain
    
    def entropy(self, y):
        """function to compute entropy"""

        class_label = np.unique(y)
        entropy = 0 
        for cls in class_label:
            p_cls = len(y[y==cls]) / len(y)
            entropy += -p_cls*np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        """function to compute gini index"""

        class_label = np.unique(y)
        gini = 0
        for cls in class_label:
            p_cls = len(y[y==cls])/len(y)
            gini += p_cls**2
        return 1-gini
    
    def calculate_leaf_vales(self,Y):
        """function to compute leaf mode"""
        Y = list(Y)
        return max(Y, key=Y.count) #XXX
    
    def print_tree(self, tree=None, indent=" "):
        """function to print the tree""" #XXX

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%selft:" % (indent),end="")
            self.print_tree(tree.left,indent + indent)
            print("%sright:" % (indent),end="")
            self.print_tree(tree.right,indent + indent)

    def fit(self, X, Y):
        """function to train the tree"""

        dataset = np.concatenate((X,Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        """function to predict new dataset"""

        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        """function to predict a single data point"""

        if tree.value != None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)